In [19]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import matthews_corrcoef

genetic_df = pd.read_csv('../data/URIEL_Phylogeny_Labels.csv', index_col=0)
df = pd.read_csv('../data/URIELPlus_Union.csv', index_col=0)
subset_df = df[df.index.isin(genetic_df.index)]

In [23]:
y_encoded = LabelEncoder().fit_transform(genetic_df['family'].to_numpy())
mi = mutual_info_classif(subset_df, y_encoded, discrete_features=True)
phi_matrix = np.ones((800, 800))
for i in range(800):
    for j in range(800):
        if i != j:
            phi_matrix[i, j] = matthews_corrcoef(subset_df.iloc[:,i], subset_df.iloc[:,j])

In [59]:
selected_idx = np.argmax(mi)
remaining = np.arange(800)
remaining = np.delete(remaining, np.argwhere(remaining == selected_idx))
curr_feat_phi = phi_matrix[remaining,selected_idx]
scores = mi[remaining] / curr_feat_phi
new_idx = remaining[np.argmax(scores)]
remaining = np.delete(remaining, np.argwhere(remaining == new_idx))
selected_idx = np.array([selected_idx, new_idx])
for i in range(3, 701):
    curr_feat_phi = phi_matrix[remaining]
    curr_feat_phi = curr_feat_phi[:, selected_idx].mean(axis=1)
    scores = mi[remaining] / curr_feat_phi
    new_idx = remaining[np.argmax(scores)]
    remaining = np.delete(remaining, np.argwhere(remaining == new_idx))
    selected_idx = np.append(selected_idx, new_idx)
    if i % 100 == 0:
        df.iloc[:,selected_idx].to_csv(f'../selection_result/base_mi_{i}.csv')
        # print(i, selected_idx)
    